In [4]:
# build_similarity_test.py
import json
import pickle
from pathlib import Path
import sys
import os

# Fix the path to load modules correctly from notebook
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # Go up one level from notebooks/ to project root
src_path = project_root / "src"

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Project root: {project_root}")
print(f"Src path: {src_path}")
print(f"Src exists: {src_path.exists()}")

from vctp.data.preprocess.object_similarity.similarity_builder import ObjectSimilarityBuilder
from vctp.data.preprocess.object_similarity.metrics import AnswerSimilarityMetric
from vctp.data.preprocess.object_similarity.aokvqa_processor import AOKVQASimilarityProcessor

# Paths (relative to project root)
annotations_dir = project_root / "data/raw/aokvqa_annotations"
sg_path = project_root / "data/processed/input_text/scene_graph_text"
output_path = project_root / "data/processed/object_similarity/train_object_select_aokvqa_answer.pkl"

print("\n=== STEP 1: Load annotations ===")
processor = AOKVQASimilarityProcessor(str(annotations_dir))
questions, answers, rationales = processor.load_split("train")

print(f"Loaded {len(questions)} questions")
print(f"Sample keys: {list(questions.keys())[:3]}")

print("\n=== STEP 2: Check scene graph files ===")
sg_dir = sg_path / "scene_graph_coco17"
sg_attr_dir = sg_path / "scene_graph_coco17_attr"

print(f"SG dir exists: {sg_dir.exists()}")
print(f"SG attr dir exists: {sg_attr_dir.exists()}")

if sg_dir.exists():
    sg_files = list(sg_dir.glob("*.json"))
    print(f"Found {len(sg_files)} scene graph files")
    if sg_files:
        print(f"Sample files: {[f.name for f in sg_files[:3]]}")
else:
    print("❌ Scene graph directory NOT FOUND!")
    print("This is why the PKL file is empty - no scene graphs to process!")
    raise FileNotFoundError(f"Scene graph directory not found: {sg_dir}")

print("\n=== STEP 3: Test with first sample ===")
# Get first key
first_key = list(questions.keys())[0]
img_id = int(first_key.split("<->")[0])

sg_file = sg_dir / f"{str(img_id).zfill(12)}.json"
print(f"Looking for scene graph: {sg_file}")
print(f"Exists: {sg_file.exists()}")

if not sg_file.exists():
    print(f"\n❌ Scene graph for image {img_id} NOT FOUND!")
    print(f"This is the problem - your test sample images don't have scene graphs!")
    print(f"\nYou need to either:")
    print(f"1. Generate scene graphs for your test images (0-9)")
    print(f"2. Use a different set of training images that have scene graphs")
    raise FileNotFoundError(f"Scene graph not found: {sg_file}")

print("\n=== STEP 4: Build similarity (if scene graphs exist) ===")
metric = AnswerSimilarityMetric()
builder = ObjectSimilarityBuilder(
    sg_dir=sg_dir,
    sg_attr_dir=sg_attr_dir,
    metric=metric,
)

similarity_dict = builder.build(
    questions=questions,
    answers=answers,
    rationales=rationales,
    output_path=str(output_path),
    show_progress=True,
)

print(f"\n✓ Built similarity dict with {len(similarity_dict)} keys")
print(f"✓ Saved to {output_path}")

# Verify
with open(output_path, "rb") as f:
    loaded = pickle.load(f)
print(f"✓ Verification: Loaded {len(loaded)} keys from PKL file")

Project root: e:\AIO\Project\VisualCoT
Src path: e:\AIO\Project\VisualCoT\src
Src exists: True


d:\Asus\App\miniconda\workspace\envs\vctp-nle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



=== STEP 1: Load annotations ===
Loaded 10 questions
Sample keys: ['0<->22MexNkBPpdZGX6sxbxVBH', '1<->22ZAvqke8EhGDj8e4eyios', '2<->22oou4R5ejh4Ay9UQG7yGT']

=== STEP 2: Check scene graph files ===
SG dir exists: True
SG attr dir exists: True
Found 1 scene graph files
Sample files: ['000000000000.json']

=== STEP 3: Test with first sample ===
Looking for scene graph: e:\AIO\Project\VisualCoT\data\processed\input_text\scene_graph_text\scene_graph_coco17\000000000000.json
Exists: True

=== STEP 4: Build similarity (if scene graphs exist) ===
[CLIPManager] Loading CLIP with tokenizer: openai/clip-vit-base-patch16 on cpu
[CLIPManager] ✓ Model cached: openai/clip-vit-base-patch16_tokenizer_cpu


100%|██████████| 10/10 [00:00<00:00, 26.98it/s]

Saved similarity scores to e:\AIO\Project\VisualCoT\data\processed\object_similarity\train_object_select_aokvqa_answer.pkl

✓ Built similarity dict with 1 keys
✓ Saved to e:\AIO\Project\VisualCoT\data\processed\object_similarity\train_object_select_aokvqa_answer.pkl
✓ Verification: Loaded 1 keys from PKL file


In [5]:
# Verify the PKL content
import pickle
from pathlib import Path

pkl_path = project_root / "data/processed/object_similarity/train_object_select_aokvqa_answer.pkl"

with open(pkl_path, "rb") as f:
    pkl_data = pickle.load(f)

print(f"=== PKL FILE CONTENT ===")
print(f"Number of keys: {len(pkl_data)}")
print(f"\nKeys in PKL:")
for key in pkl_data.keys():
    print(f"  - {key}")

# Show first entry details
if pkl_data:
    first_key = list(pkl_data.keys())[0]
    print(f"\n=== First Entry Details ===")
    print(f"Key: {first_key}")
    print(f"Value type: {type(pkl_data[first_key])}")
    print(f"Value: {pkl_data[first_key]}")
    
    # If it's a dict, show its structure
    if isinstance(pkl_data[first_key], dict):
        print(f"\nValue has {len(pkl_data[first_key])} objects:")
        for obj_name, score in list(pkl_data[first_key].items())[:5]:
            print(f"  {obj_name}: {score:.4f}")

=== PKL FILE CONTENT ===
Number of keys: 1

Keys in PKL:
  - 0<->22MexNkBPpdZGX6sxbxVBH

=== First Entry Details ===
Key: 0<->22MexNkBPpdZGX6sxbxVBH
Value type: <class 'dict'>
Value: {'arm': 0.5491220951080322, 'face': 0.6824137568473816, 'hair': 0.652134120464325, 'hand': 0.5920497179031372, 'handle': 0.6108238697052002, 'head': 0.5802187323570251, 'leg': 0.5364360213279724, 'man': 0.651434063911438, 'person': 0.649359941482544, 'player': 0.7496165037155151, 'racket': 0.6210474967956543, 'shirt': 0.6177270412445068, 'shoe': 0.6860548257827759, 'short': 0.6840023994445801, 'sneaker': 0.6612303256988525, 'sock': 0.6086715459823608, 'shadow': 0.6272501945495605, 'ball': 0.6987450122833252, 'line': 0.5646349787712097, 'court': 0.6837835311889648, 'knee': 0.5892088413238525, 'watch': 0.6974292993545532, 'ground': 0.6758285164833069, 'mouth': 0.6402465105056763, 'wrist': 0.5146611332893372, 'writing': 0.6421564817428589, 'tennis ball': 0.5933190584182739, 'tennis racket': 0.5826085209846497

In [ ]:
# Compare with original implementation
import json

# Load the annotation to see the actual question/answer
with open(project_root / "data/raw/aokvqa_annotations/aokvqa_v1p0_train_from_hf.json", "r") as f:
    train_data = json.load(f)

# Find the sample with image_id=0
sample = train_data[0]
print("=== ORIGINAL SAMPLE ===")
print(f"Image ID: {sample['image_id']}")
print(f"Question ID: {sample['question_id']}")
print(f"Question: {sample['question']}")
print(f"Choices: {sample['choices']}")
print(f"Correct choice: {sample['correct_choice_idx']}")
print(f"Direct answers: {sample['direct_answers']}")

# Get the answer
answer = sample['choices'][sample['correct_choice_idx']]
print(f"\n=== ANSWER ===")
print(f"Answer: {answer}")

# Now check the PKL scores
print(f"\n=== OBJECT SIMILARITY TO ANSWER '{answer}' ===")
obj_scores = pkl_data['0<->22MexNkBPpdZGX6sxbxVBH']
sorted_objs = sorted(obj_scores.items(), key=lambda x: x[1], reverse=True)

print(f"\nTop 10 most relevant objects:")
for i, (obj_name, score) in enumerate(sorted_objs[:10], 1):
    print(f"  {i}. {obj_name}: {score:.4f}")

print(f"\nBottom 5 least relevant objects:")
for obj_name, score in sorted_objs[-5:]:
    print(f"  {obj_name}: {score:.4f}")



=== ORIGINAL SAMPLE ===
Image ID: 0
Question ID: 22MexNkBPpdZGX6sxbxVBH
Question: What is the man by the bags awaiting?
Choices: ['skateboarder', 'train', 'delivery', 'cab']
Correct choice: 3
Direct answers: ['ride', 'ride', 'bus', 'taxi', 'travelling', 'traffic', 'taxi', 'cab', 'cab', 'his ride']

=== ANSWER ===
Answer: cab

=== OBJECT SIMILARITY TO ANSWER 'cab' ===

Top 10 most relevant objects:
  1. player: 0.7496
  2. ball: 0.6987
  3. watch: 0.6974
  4. shoe: 0.6861
  5. short: 0.6840
  6. court: 0.6838
  7. face: 0.6824
  8. ground: 0.6758
  9. sneaker: 0.6612
  10. hair: 0.6521

Bottom 5 least relevant objects:
  head: 0.5802
  line: 0.5646
  arm: 0.5491
  leg: 0.5364
  wrist: 0.5147


In [9]:
# Test loading and using the PKL file directly
import pickle
from typing import List
print("\n=== TEST OBJECT SIMILARITY RETRIEVAL ===")

# Load PKL
pkl_path = project_root / "data/processed/object_similarity/train_object_select_aokvqa_answer.pkl"
with open(pkl_path, "rb") as f:
    object_similarity_dict = pickle.load(f)

print(f"✓ Loaded {len(object_similarity_dict)} entries from PKL")

# Function to get top K objects for a question
def get_top_objects(question_key: str, top_k: int = 10) -> List[str]:
    """Get top K most relevant objects for a question."""
    if question_key not in object_similarity_dict:
        return []
    
    obj_scores = object_similarity_dict[question_key]
    # Sort by score descending
    sorted_objs = sorted(obj_scores.items(), key=lambda x: x[1], reverse=True)
    # Return top K object names
    return [obj_name for obj_name, score in sorted_objs[:top_k]]

# Test with first sample
test_key = '0<->22MexNkBPpdZGX6sxbxVBH'
top_objects = get_top_objects(test_key, top_k=10)

print(f"\n✓ Retrieved top 10 objects for '{test_key}':")
for i, obj_name in enumerate(top_objects, 1):
    score = object_similarity_dict[test_key][obj_name]
    print(f"  {i}. {obj_name}: {score:.4f}")

print("\n✓ PKL file works correctly for retrieval!")


=== TEST OBJECT SIMILARITY RETRIEVAL ===
✓ Loaded 1 entries from PKL

✓ Retrieved top 10 objects for '0<->22MexNkBPpdZGX6sxbxVBH':
  1. player: 0.7496
  2. ball: 0.6987
  3. watch: 0.6974
  4. shoe: 0.6861
  5. short: 0.6840
  6. court: 0.6838
  7. face: 0.6824
  8. ground: 0.6758
  9. sneaker: 0.6612
  10. hair: 0.6521

✓ PKL file works correctly for retrieval!


In [4]:
# Fix đường dẫn trong notebook
import pickle
from pathlib import Path

# ĐÚNG: Get project root (lên 1 cấp từ notebooks/)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # Go up from notebooks/ to project root

print(f"Notebook dir: {notebook_dir}")
print(f"Project root: {project_root}")

# Đường dẫn đến PKL file
pkl_path = project_root / "data/processed/object_similarity/train_object_select_aokvqa_answer.pkl"
print(f"PKL path: {pkl_path}")
print(f"PKL exists: {pkl_path.exists()}")

# Load existing
with open(pkl_path, "rb") as f:
    existing = pickle.load(f)

print(f"\nExisting PKL has {len(existing)} keys")

# Get sample 0 data
sample_0_objects = existing["0<->22MexNkBPpdZGX6sxbxVBH"]
print(f"Sample 0 has {len(sample_0_objects)} objects")

# All 10 keys
keys = [
    "0<->22MexNkBPpdZGX6sxbxVBH",
    "1<->22ZAvqke8EhGDj8e4eyios", 
    "2<->22oou4R5ejh4Ay9UQG7yGT",
    "3<->22qCSTGL82TcgGtu9wtcrL",
    "4<->22qvnfBREuA4LUGw3cfTgg",
    "5<->23NaDk4gncPVsGzZ7UuxNK",
    "6<->23VQ24vGdCfx2vuwoNqxwk",
    "7<->23ibLk5tF4wVKd7MMms9Yp",
    "8<->23x4iQXWfAeZxqCJVbTuv5",
    "9<->23xWxgFhSv5sZaxwJrqJwm",
]

# Create dummy with all samples using sample 0's data
dummy = {key: sample_0_objects.copy() for key in keys}

# Save
with open(pkl_path, "wb") as f:
    pickle.dump(dummy, f)

print(f"\n✓ Created dummy PKL with {len(dummy)} samples")
print(f"✓ Saved to {pkl_path}")

# Verify
with open(pkl_path, "rb") as f:
    loaded = pickle.load(f)
    
print(f"\n=== VERIFICATION ===")
print(f"Loaded PKL has {len(loaded)} keys")
print(f"Keys:")
for i, key in enumerate(loaded.keys(), 1):
    print(f"  {i}. {key}")
print(f"\nAll samples have {len(loaded[keys[0]])} objects")

Notebook dir: e:\AIO\Project\VisualCoT\notebooks
Project root: e:\AIO\Project\VisualCoT
PKL path: e:\AIO\Project\VisualCoT\data\processed\object_similarity\train_object_select_aokvqa_answer.pkl
PKL exists: True

Existing PKL has 1 keys
Sample 0 has 28 objects

✓ Created dummy PKL with 10 samples
✓ Saved to e:\AIO\Project\VisualCoT\data\processed\object_similarity\train_object_select_aokvqa_answer.pkl

=== VERIFICATION ===
Loaded PKL has 10 keys
Keys:
  1. 0<->22MexNkBPpdZGX6sxbxVBH
  2. 1<->22ZAvqke8EhGDj8e4eyios
  3. 2<->22oou4R5ejh4Ay9UQG7yGT
  4. 3<->22qCSTGL82TcgGtu9wtcrL
  5. 4<->22qvnfBREuA4LUGw3cfTgg
  6. 5<->23NaDk4gncPVsGzZ7UuxNK
  7. 6<->23VQ24vGdCfx2vuwoNqxwk
  8. 7<->23ibLk5tF4wVKd7MMms9Yp
  9. 8<->23x4iQXWfAeZxqCJVbTuv5
  10. 9<->23xWxgFhSv5sZaxwJrqJwm

All samples have 28 objects
